In [45]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
import datetime
import cv2
import statsmodels
from statsmodels.tsa.ar_model import AR

In [2]:
plot = plt.plot

In [241]:
popWtrData = pd.read_csv('county_pop_well.csv')

cty = np.unique(popWtrData.COUNTY_NAME)

MAX_MODEL_LENGTH = 20

iCty = np.where(popWtrData.COUNTY_NAME == 'san diego')

predErr_All = np.zeros((len(cty),MAX_MODEL_LENGTH))
for iCty in range(len(cty)):#
    
    idxCty = popWtrData.COUNTY_NAME == cty[iCty]
    
    numEntries = np.sum(popWtrData.COUNTY_NAME==cty[iCty])
    
    if numEntries <= MAX_MODEL_LENGTH+5:
        continue
    
    # Predict
    for iML,mdlLen in enumerate(np.arange(1,MAX_MODEL_LENGTH)):
        
        #create train/test datasets
        X = np.array(popWtrData.GSE_WSE[idxCty].dropna())
        train_data = X[1:len(X)-MAX_MODEL_LENGTH]
        test_data = np.array([X[len(X)-MAX_MODEL_LENGTH+mdlLen]])
        #test_data = X[len(X)-mdlLen:]
        tmpPop = np.array(popWtrData.sum_pop[idxCty])[len(X)-MAX_MODEL_LENGTH+mdlLen]
        
        #train the autoregression model
        model = AR(train_data)
        model_fitted = model.fit()
        
        if 0:
            print('The lag value chose is: %s' % model_fitted.k_ar)
            print('The coefficients of the model are:\n %s' % model_fitted.params)


        predictions = model_fitted.predict(
            start=len(train_data), 
            end=len(train_data) + len(test_data)-1, 
            dynamic=False)

        # Find the relative error
        tmp = np.linalg.norm(predictions-test_data)/np.linalg.norm(test_data)*100
        #tmp = test_data
        #
        predErr_All[iCty,iML] = tmp

if 0:
    plt.plot(predictions)
    plt.plot(test_data)
    plt.show()

In [253]:
x = pd.DataFrame(data=predErr_All,index=cty,columns=np.arange(2000,2020))

In [254]:
x.to_csv('normError.csv')